In [31]:
from pySWATPlus.TxtinoutReader import TxtinoutReader
from pathlib import Path
import random
from SALib.sample import saltelli
from SALib.analyze import sobol
import numpy as np
import random
import concurrent.futures

In [32]:
random.seed(42)  # Set seed for reproducibility

In [33]:
cwd = Path.cwd()
txtinout_folder_path = Path(cwd / 'muga')   #replace with the path to the SWAT+ project folder

In [34]:
txtinout_reader = TxtinoutReader(txtinout_folder_path)

In [35]:
txtinout_reader.set_beginning_and_end_year(2010, 2012)
txtinout_reader.set_warmup(1)
txtinout_reader.enable_object_in_print_prt('channel_sd', True, False, False, False)


In [36]:
#define the problem: in our case, we just want to calibrate epco and esco and evaluate the model

def run_and_evaluate_swat(epco: int = 0.5, esco: int = 0.5):
    
    print(f'Running SWAT with epco = {epco} and esco = {esco} \n')
    
    #copy swat so simulations don't interfere with each other and can run in parallel
    tmp_path = txtinout_reader.copy_swat()
    
    reader = TxtinoutReader(tmp_path)
    hydrology_hyd = reader.register_file('hydrology.hyd', has_units = False, )
    hydrology_hyd_df = hydrology_hyd.df
    
    #overwrite the values of epco and esco
    hydrology_hyd_df['epco'] = epco
    hydrology_hyd_df['esco'] = esco
    
    #save the changes
    hydrology_hyd.overwrite_file()
    
    #run the model
    txt_in_out_result = reader.run_swat(show_output=False)
    
    #read the results
    reader = TxtinoutReader(txt_in_out_result)    
    channel_sdmorph = reader.register_file('channel_sdmorph_day.txt', has_units = True, )
    channel_sdmorph_df = channel_sdmorph.df
    
    #here, you should read your observations and calculate the objective function
    #for now, let's just return a mock value
    return random.random()

# Wrapper function for parallel execution
def evaluate(params):
    return run_and_evaluate_swat(*params)

In [37]:
# Define the model inputs
problem = {
  'num_vars': 2,
  'names': ['epco', 'esco'],
  'bounds': [[0, 1]]*2
}

In [38]:
# sample
param_values = saltelli.sample(problem, 2)

# Parallel execution
with concurrent.futures.ProcessPoolExecutor() as executor:
    y = np.array(list(executor.map(evaluate, param_values)))

sobol_indices = sobol.analyze(problem, y)


/tmp/ipykernel_810609/950771132.py:2: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 2)


Running SWAT with epco = 0.46875 and esco = 0.46875 
Running SWAT with epco = 0.09375 and esco = 0.65625 
Running SWAT with epco = 0.46875 and esco = 0.46875 
Running SWAT with epco = 0.46875 and esco = 0.65625 
Running SWAT with epco = 0.09375 and esco = 0.46875 
Running SWAT with epco = 0.59375 and esco = 0.15625 
Running SWAT with epco = 0.96875 and esco = 0.96875 
Running SWAT with epco = 0.59375 and esco = 0.96875 
Running SWAT with epco = 0.96875 and esco = 0.96875 

Running SWAT with epco = 0.59375 and esco = 0.15625 
Running SWAT with epco = 0.96875 and esco = 0.15625 



Running SWAT with epco = 0.09375 and esco = 0.65625 








